In [1]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow as tf

from tensorflow.random import set_seed
from tensorflow.keras.preprocessing import image_dataset_from_directory as im_ds_from_dir
from tensorflow.keras.metrics import MeanSquaredError as MSE
from tensorflow import boolean_mask
from sklearn.model_selection import train_test_split

import numpy as np
import cv2 as cv
import sklearn.neighbors as sk

from active_thresh_segseg import segseg
from microscope_image_preprocessing import load_image

#PATH TO DIR OR FILE
PATH = '2024-07-29'

num_bacteria = 3

In [2]:
cuts = []
num = 0
for path in load_image(PATH):
    
    cuts += segseg(path,interpol=cv.INTER_AREA, dst_size=32)
    

['2024-07-29\\1-1.bmp', '2024-07-29\\1-10.bmp', '2024-07-29\\1-11.bmp', '2024-07-29\\1-12.bmp', '2024-07-29\\1-13.bmp', '2024-07-29\\1-14.bmp', '2024-07-29\\1-15.bmp', '2024-07-29\\1-16.bmp', '2024-07-29\\1-2.bmp', '2024-07-29\\1-3.bmp', '2024-07-29\\1-4.bmp', '2024-07-29\\1-5.bmp', '2024-07-29\\1-6.bmp', '2024-07-29\\1-7.bmp', '2024-07-29\\1-8.bmp', '2024-07-29\\1-9.bmp']
0.1121513843536377
0.0794374942779541
0.06200098991394043
0.06350493431091309
0.04454159736633301
0.0465242862701416
0.042999982833862305
0.04252767562866211
0.04500532150268555
0.04753541946411133
0.04100298881530762
0.04800558090209961
0.061248779296875
0.05550956726074219
0.04399728775024414
0.04701495170593262


In [3]:
for i in range(len(cuts)):
    cuts[i] = (cuts[i],cuts[i])
print(len(cuts))


2415


In [4]:
tr, ts = train_test_split(cuts,test_size=0.1,shuffle=False)
print(len(tr))
print(len(ts))

image_dir='images/minis'
target_dir='images/minis'
target_size=(32,32) # set this to the target size you want
channels=3 # for color images
color_mode='grayscale'
shuffle=True,
seed=123
class_mode=None
batch_size=32 # set this to desired batch size
vsplit=.2 # set this to the validation split you want
image_gen=im_ds_from_dir(image_dir, color_mode=color_mode, batch_size=batch_size,image_size=target_size,seed=seed,subset='training',
                         validation_split=vsplit, labels=None,data_format='channels_last')
val_gen=im_ds_from_dir(image_dir, color_mode=color_mode, batch_size=batch_size,image_size=target_size,seed=seed,subset='validation',
                         validation_split=vsplit, labels=None,data_format='channels_last')
norm_layer = tf.keras.layers.Rescaling(1./255)
image_gen = image_gen.map(lambda x: (norm_layer(x) - 1e-6,norm_layer(x) - 1e-6))
val_gen = val_gen.map(lambda x: (norm_layer(x) - 1e-6,norm_layer(x) - 1e-6))

for x in image_gen.unbatch().as_numpy_iterator():
    print(x[1])
    break

2173
242
Found 182 files.
Using 146 files for training.
Found 182 files.
Using 36 files for validation.
[[[0.8470579 ]
  [0.85097945]
  [0.8431363 ]
  ...
  [0.8313716 ]
  [0.8431363 ]
  [0.83921474]]

 [[0.8588226 ]
  [0.8588226 ]
  [0.85097945]
  ...
  [0.82352847]
  [0.8352932 ]
  [0.8313716 ]]

 [[0.8666657 ]
  [0.8666657 ]
  [0.8588226 ]
  ...
  [0.8196069 ]
  [0.8313716 ]
  [0.8313716 ]]

 ...

 [[0.8039206 ]
  [0.79999906]
  [0.78823435]
  ...
  [0.7843128 ]
  [0.78823435]
  [0.7843128 ]]

 [[0.8196069 ]
  [0.81568533]
  [0.8078422 ]
  ...
  [0.7921559 ]
  [0.7960775 ]
  [0.7921559 ]]

 [[0.8313716 ]
  [0.82745004]
  [0.82352847]
  ...
  [0.8039206 ]
  [0.8078422 ]
  [0.8078422 ]]]


a = tf.data.experimental.from_list(tr)
dataset_length = [i for i,_ in enumerate(a)][-1] + 1
print(dataset_length)

norm_layer = tf.keras.layers.Rescaling(1./255)
im_b = a.map(lambda x,y:(norm_layer(x), norm_layer(y)))

for i in im_b.as_numpy_iterator():
    print(i[0].shape)
    break

In [5]:
encoder = keras.Sequential(
    [
        # layers.Conv2D(16,1, activation='relu', padding='same'),
        # layers.Conv2D(16,1,activation='relu', padding='same'),
        # layers.MaxPooling2D((2,2), padding='same'),

        layers.Conv2D(32,3, activation='relu', padding='same'),
        layers.Conv2D(32,3,activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same', strides=2),#16

        layers.Conv2D(64,(3,3), activation='relu', padding='same'),
        layers.Conv2D(64,(3,3),activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same',strides=2),#8

        layers.Conv2D(128,(3,3), activation='relu', padding='same'),
        layers.Conv2D(128,(3,3),activation='relu', padding='same'),
        layers.MaxPooling2D((2,2), padding='same',strides=2),#4

        layers.Conv2D(128,(3,3), activation='relu', padding='same'),
        layers.Conv2D(128,(3,3),activation='relu', padding='same'),

    ]
)

flattenOut = keras.Sequential(
    [
        layers.Flatten()
    ]
)

decoder = keras.Sequential(
    [
        layers.UpSampling2D((2,2)), #2^2
        layers.Conv2D(64,(3,3), activation='relu', padding='same'),#4^2
        layers.Conv2D(32,(3,3),activation='relu', padding='same'),#6^2
        
        layers.UpSampling2D((2,2)),#12^2
        layers.Conv2D(8,(3,3), activation='relu', padding='same'),#14^2
        layers.Conv2D(1,(3,3),activation='relu', padding='same'),#16^2
        
        layers.UpSampling2D((2,2))#32
    ]
)

In [6]:
def conv(layer, channel_num):
    layer = layers.Conv2D(channel_num, 3, padding='same', activation='tanh')(layer)
    layer = layers.Conv2D(channel_num, 3, padding='same', activation='tanh')(layer)
    return layer

def downconv(layer, channel_num, dropout):
    c = conv(layer, channel_num)
    mp = layers.MaxPooling2D(2)(c)
    layer = layers.Dropout(dropout)(mp)
    return c, layer

def upconv(layer, skip, channel_num, dropout):
    layer = layers.Conv2DTranspose(channel_num, 3, 2, padding='same', activation='tanh')(layer)
    layer = layers.concatenate([layer, skip])
    layer = layers.Dropout(dropout)(layer)
    layer = conv(layer, channel_num)
    return layer

In [7]:
input_layer = tf.keras.Input(shape=(32,32,1))

skip1, encode2 = downconv(input_layer,32,0.1)
skip2, encode2 = downconv(encode2, 64, 0.2)
skip3, encode2 = downconv(encode2, 128, 0.25)
bottleneck = conv(encode2, 256)
encode2 = upconv(bottleneck, skip3, 128, 0.25)
encode2 = upconv(encode2, skip2, 64, 0.2)
encode2 = upconv(encode2, skip1, 32, 0.1)
autoencode = keras.layers.Conv2D(8, 3, padding='same', activation='relu')(encode2)
autoencode = keras.layers.Conv2D(1, 3, padding='same', activation='sigmoid')(autoencode)


enc = encoder(input_layer)
autoencoder = decoder(enc)

# encoder.summary()
encenc = keras.Model(inputs=input_layer,outputs=autoencode)
encenc.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 32, 32,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │      9,248 │ conv2d_12[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_13[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16, 16,    │          0 │ max_pooling2d_3[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 16, 16,    │     18,496 │ dropout[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 16, 16,    │     36,928 │ conv2d_14[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 8, 8, 64)  │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 8, 8, 64)  │          0 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 8, 8, 128) │     73,856 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 8, 8, 128) │    147,584 │ conv2d_16[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 4, 4, 128) │          0 │ conv2d_17[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 4, 4, 128) │          0 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 4, 4, 256) │    295,168 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 4, 4, 256) │    590,080 │ conv2d_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 8, 8, 128) │    295,040 │ conv2d_19[0][0]   │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8, 8, 256) │          0 │ conv2d_transpose… │
│ (Concatenate)       │                   │            │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 8, 8, 256) │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 8, 8, 128) │    295,040 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 2,142,417 (8.17 MB)

 Trainable params: 2,142,417 (8.17 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
SEED = 1
set_seed(SEED)

model = keras.Model(inputs=input_layer,
                    outputs=autoencoder)

#AE trainloop

model.summary()
# model.trainable = False


bottleneck = keras.Model(inputs= input_layer, outputs = [model.layers[1].output])
model2 = keras.Sequential()
model2.add(model.layers[1])
model2.add(layers.Dense(100))
model2.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 4, 4, 128)      │       581,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 32, 32, 1)      │        94,641 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 676,241 (2.58 MB)

 Trainable params: 676,241 (2.58 MB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 4, 4, 128)      │       581,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 581,600 (2.22 MB)

 Trainable params: 581,600 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', restore_best_weights=True, patience=20, start_from_epoch=15)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
epochs = 100

# md_clone = model

# model.compile(optimizer=optimizer, loss=MSE)

# md_clone.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

# model.fit(image_gen, epochs=epochs,callbacks=callbacks,batch_size=batch_size, validation_data=val_gen)

# print(model.predict(image_gen))



Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch

In [10]:
a = model.predict(next(iter(image_gen))[0])
print(a.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
(32, 32, 32, 1)


In [11]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', restore_best_weights=True, patience=20, start_from_epoch=0)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
encenc.compile(optimizer=optimizer, loss="mse")
encenc.fit(image_gen, epochs=epochs,callbacks=callbacks,batch_size=batch_size, validation_data=val_gen)

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 221ms/step - loss: 0.0514 - val_loss: 0.0123
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 0.0112 - val_loss: 0.0085
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - loss: 0.0084 - val_loss: 0.0066
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - loss: 0.0065 - val_loss: 0.0054
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - loss: 0.0052 - val_loss: 0.0043
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - loss: 0.0044 - val_loss: 0.0037
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - loss: 0.0032 - val_loss: 0.0046
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - loss: 0.0041 - val_loss: 0.0027
Epoch 10/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 11/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - loss: 0.0029 - val_loss: 0.0021
Epoch 12/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - loss: 0.0

In [12]:
encenc.predict(next(iter(image_gen))[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step


array([[[[0.80982447],
         [0.81889385],
         [0.8235971 ],
         ...,
         [0.8376715 ],
         [0.8390603 ],
         [0.8146501 ]],

        [[0.802322  ],
         [0.7959465 ],
         [0.81556755],
         ...,
         [0.8513219 ],
         [0.8457181 ],
         [0.8318788 ]],

        [[0.82474095],
         [0.822365  ],
         [0.8090024 ],
         ...,
         [0.85741997],
         [0.86414677],
         [0.84554   ]],

        ...,

        [[0.7880685 ],
         [0.78751975],
         [0.7941035 ],
         ...,
         [0.7907539 ],
         [0.79419655],
         [0.8073244 ]],

        [[0.7811362 ],
         [0.7844043 ],
         [0.7812548 ],
         ...,
         [0.79266655],
         [0.8069253 ],
         [0.8094416 ]],

        [[0.7876475 ],
         [0.8006008 ],
         [0.7890015 ],
         ...,
         [0.7970282 ],
         [0.8054349 ],
         [0.803827  ]]],


       [[[0.8061462 ],
         [0.8156267 ],
         [0.82

In [13]:
encenc.trainable
print(next(iter(image_gen.unbatch()))[0])
print(encenc.summary())

tf.Tensor(
[[[0.82352847]
  [0.8196069 ]
  [0.81568533]
  ...
  [0.8588226 ]
  [0.8666657 ]
  [0.8705873 ]]

 [[0.8196069 ]
  [0.81568533]
  [0.81176376]
  ...
  [0.8470579 ]
  [0.854901  ]
  [0.8588226 ]]

 [[0.81176376]
  [0.8078422 ]
  [0.79999906]
  ...
  [0.8352932 ]
  [0.8431363 ]
  [0.8470579 ]]

 ...

 [[0.8039206 ]
  [0.8078422 ]
  [0.7960775 ]
  ...
  [0.8352932 ]
  [0.8431363 ]
  [0.8431363 ]]

 [[0.8078422 ]
  [0.81176376]
  [0.79999906]
  ...
  [0.8431363 ]
  [0.85097945]
  [0.8470579 ]]

 [[0.8078422 ]
  [0.81176376]
  [0.8039206 ]
  ...
  [0.8470579 ]
  [0.8588226 ]
  [0.854901  ]]], shape=(32, 32, 1), dtype=float32)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 32, 32,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 32, 32,    │      9,248 │ conv2d_12[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_13[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16, 16,    │          0 │ max_pooling2d_3[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 16, 16,    │     18,496 │ dropout[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 16, 16,    │     36,928 │ conv2d_14[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 8, 8, 64)  │          0 │ conv2d_15[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 8, 8, 64)  │          0 │ max_pooling2d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 8, 8, 128) │     73,856 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 8, 8, 128) │    147,584 │ conv2d_16[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 4, 4, 128) │          0 │ conv2d_17[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 4, 4, 128) │          0 │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 4, 4, 256) │    295,168 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 4, 4, 256) │    590,080 │ conv2d_18[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 8, 8, 128) │    295,040 │ conv2d_19[0][0]   │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8, 8, 256) │          0 │ conv2d_transpose… │
│ (Concatenate)       │                   │            │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 8, 8, 256) │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 8, 8, 128) │    295,040 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 6,427,253 (24.52 MB)

 Trainable params: 2,142,417 (8.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,284,836 (16.35 MB)

None


In [5]:
import tensorflow as tf
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [6]:
print(tf.__version__)

2.16.2


In [1]:
i = 0


In [2]:
print(i)

0


thing goes here